In [1]:
!mkdir ~/dengue/af3_models

In [6]:
fusion_seq = 'MADLELERAADVKWEDQAEISGSSPILSITISEDGSMSIKNEEEEQILGGGGSGGGGAGVLWDVPSPPPMGKAELEDGAYRIKQKGILGYSQIGAGVYKEGTFHTMWHVTRGAVLMHKGKRIEPSWADVKKDLISYGGGWKLEGEWKEGEEVQVLALEPGKNPRAVQTKPGLFKTNAGTIGAVSLDFSPGTSGSPIIDKKGKVVGLYGNGVVTRSGAYVSAIAQTEKSIEDNPEIEDDIFRK'

import json
from pathlib import Path

name = 'fusion'
sequence = fusion_seq

prot_sequence = dict(id='A', sequence=sequence, modifications=[], unpairedMsa=None, pairedMsa=None)
# zn_sequence = dict(id='Z', ccdCodes=['ZN'])

job = dict(name=name,
           modelSeeds=[0,],# 123, 666, 888, 314159],
           sequences=[dict(protein=prot_sequence),
                     ],
           dialect='alphafold3',
           version=2
          )

path = Path(f'~/dengue/af3_models/{name}.json').expanduser()
print(f'''python run_alphafold.py \
    --json_path={path} \
    --output_dir={path.parent} \
    --db_dir=/tmp/public_databases \
    --model_dir=/tmp/af3_weights;''')
path.write_text( json.dumps(job) )


python run_alphafold.py     --json_path=/opt/xchem-fragalysis-2/mferla/dengue/af3_models/fusion.json     --output_dir=/opt/xchem-fragalysis-2/mferla/dengue/af3_models     --db_dir=/tmp/public_databases     --model_dir=/tmp/af3_weights;


436

In [7]:
import json
from pathlib import Path

name = 'split_full'

protA_sequence = dict(id='A', sequence='MADLELERAADVKWEDQAEISGSSPILSITISEDGSMSIKNEEEEQIL', modifications=[], unpairedMsa=None, pairedMsa=None)
protB_sequence = dict(id='B', sequence='AGVLWDVPSPPPMGKAELEDGAYRIKQKGILGYSQIGAGVYKEGTFHTMWHVTRGAVLMHKGKRIEPSWADVKKDLISYGGGWKLEGEWKEGEEVQVLALEPGKNPRAVQTKPGLFKTNAGTIGAVSLDFSPGTSGSPIIDKKGKVVGLYGNGVVTRSGAYVSAIAQTEKSIEDNPEIEDDIFRK', modifications=[], unpairedMsa=None, pairedMsa=None)

job = dict(name=name,
           modelSeeds=[0, 123, 666, 888, 314159],
           sequences=[dict(protein=protA_sequence),
                      dict(protein=protB_sequence),
                     ],
           dialect='alphafold3',
           version=2
          )

path = Path(f'~/dengue/af3_models/{name}.json').expanduser()
print(f'''python run_alphafold.py \
    --json_path={path} \
    --output_dir={path.parent} \
    --db_dir=/tmp/public_databases \
    --model_dir=/tmp/af3_weights;''')
path.write_text( json.dumps(job) )


python run_alphafold.py     --json_path=/opt/xchem-fragalysis-2/mferla/dengue/af3_models/split_full.json     --output_dir=/opt/xchem-fragalysis-2/mferla/dengue/af3_models     --db_dir=/tmp/public_databases     --model_dir=/tmp/af3_weights;


557

In [8]:
import json
from pathlib import Path

name = 'split_andre'

protA_sequence = dict(id='A', sequence='MADLELERAADVKWEDQAEISGSSPILSITISEDGSMSIKN', modifications=[], unpairedMsa=None, pairedMsa=None)
protB_sequence = dict(id='B', sequence='AELEDGAYRIKQKGILGYSQIGAGVYKEGTFHTMWHVTRGAVLMHKGKRIEPSWADVKKDLISYGGGWKLEGEWKEGEEVQVLALEPGKNPRAVQTKPGLFKTNAGTIGAVSLDFSPGTSGSPIIDKKGKVVGLYGNGVVTRSGAYVSAIAQTEKSI', modifications=[], unpairedMsa=None, pairedMsa=None)

job = dict(name=name,
           modelSeeds=[0, 123, 666, 888, 314159],
           sequences=[dict(protein=protA_sequence),
                      dict(protein=protB_sequence),
                     ],
           dialect='alphafold3',
           version=2
          )

path = Path(f'~/dengue/af3_models/{name}.json').expanduser()
print(f'''python run_alphafold.py \
    --json_path={path} \
    --output_dir={path.parent} \
    --db_dir=/tmp/public_databases \
    --model_dir=/tmp/af3_weights;''')
path.write_text( json.dumps(job) )


python run_alphafold.py     --json_path=/opt/xchem-fragalysis-2/mferla/dengue/af3_models/split_andre.json     --output_dir=/opt/xchem-fragalysis-2/mferla/dengue/af3_models     --db_dir=/tmp/public_databases     --model_dir=/tmp/af3_weights;


523

## Deviation

In [ ]:
import gemmi
import numpy as np
import json

def get_rmsd(ref_model, mob_model, residue_pairs):
    ref_positions = []
    mob_positions = []
    
    for (res1, res2) in residue_pairs:
        chain_1, resnum_1 = res1
        chain_2, resnum_2 = res2
    
        res1_atoms = ref_model[chain_1][resnum_1-1]['CA'][0]
        res2_atoms = mob_model[chain_2][resnum_2-1]['CA'][0]
        ref_positions.append(res1_atoms.pos)
        mob_positions.append(res2_atoms.pos)
    
    transformation = gemmi.superpose_positions(mob_positions, ref_positions)
    return transformation.rmsd

ref: gemmi.Model = gemmi.read_structure("af3_models/split_andre/split_andre_model.cif")[0]
data = []
info = json.load(open('af3_models/split_andre/split_andre_data.json'))

for path in Path('af3_models/split_andre').glob('*/model.cif'):
    name = path.parent.name
    query_model: gemmi.Model = gemmi.read_structure(path.as_posix())[0]
    #
    residue_pairs = []
    for chain in info['sequences']:
        if 'protein' not in chain:
            continue
        for i in range(1, len(chain['protein']['sequence'])+1):
            residue_pairs.append(((chain['protein']['id'], i), (chain['protein']['id'], i)))
    data.append({'name': name, 'experiment': 'split_andre', 'rmsd': get_rmsd(ref, query_model, residue_pairs)})


chainB_offset = len('AGVLWDVPSPPPMGK')
for path in Path('af3_models/split_full').glob('*/model.cif'):
    name = path.parent.name
    query_model: gemmi.Model = gemmi.read_structure(path.as_posix())[0]
    #
    residue_pairs = []
    for chain in info['sequences']:
        if 'protein' not in chain:
            continue
        for i in range(1, len(chain['protein']['sequence'])+1):
            residue_pairs.append(((chain['protein']['id'], i), (chain['protein']['id'], i if chain['protein']['id']=='A' else i+chainB_offset)))
    data.append({'name': name, 'experiment': 'split_full', 'rmsd': get_rmsd(ref, query_model, residue_pairs)})

chainB_offset = len('MADLELERAADVKWEDQAEISGSSPILSITISEDGSMSIKNEEEEQILGGGGSGGGGAGVLWDVPSPPPMGK')
for path in Path('af3_models/fusion').glob('*/model.cif'):
    name = path.parent.name
    query_model: gemmi.Model = gemmi.read_structure(path.as_posix())[0]
    #
    residue_pairs = []
    for chain in info['sequences']:
        if 'protein' not in chain:
            continue
        for i in range(1, len(chain['protein']['sequence'])+1):
            residue_pairs.append(((chain['protein']['id'], i), ('A', i if chain['protein']['id']=='A' else i+chainB_offset)))
    data.append({'name': name, 'experiment': 'fusion', 'rmsd': get_rmsd(ref, query_model, residue_pairs)})
    
import pandas as pd

df = pd.DataFrame(data).sort_values('rmsd')
df.to_csv('af3_rmsds.csv')